In [1]:
pip install pandas regex matplotlib sklearn

  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1304 sha256=ebcdc02b5aebfba2915ec4ea0ca5f3ec150f0264e15d7bfe763a978875adb3a7
  Stored in directory: /workspace/.pyenv_mirror/pip_cache/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd 
import regex as reg
import re
import matplotlib.pyplot as plt
import unicodedata
import nltk
import string
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

from sklearn import model_selection, svm
from sklearn.metrics import classification_report, accuracy_score
from nltk.corpus import stopwords

In [4]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv')

In [5]:
df_raw.info

<bound method DataFrame.info of                                                     url  is_spam
0     https://briefingday.us8.list-manage.com/unsubs...     True
1                                https://www.hvper.com/     True
2                    https://briefingday.com/m/v4n3i4f3     True
3      https://briefingday.com/n/20200618/m#commentform    False
4                           https://briefingday.com/fan     True
...                                                 ...      ...
2994  https://www.smartcitiesworld.net/news/news/dee...    False
2995                      https://www.youtube.com/watch     True
2996  https://techcrunch.com/2019/07/04/an-optimisti...    False
2997  https://www.technologyreview.com/2019/12/20/13...    False
2998       https://www.bbc.com/news/technology-51018758    False

[2999 rows x 2 columns]>

In [6]:
df_raw['is_spam'] = df_raw['is_spam'].apply(lambda x: 1 if x == True else 0)

In [7]:
df_raw.info

<bound method DataFrame.info of                                                     url  is_spam
0     https://briefingday.us8.list-manage.com/unsubs...        1
1                                https://www.hvper.com/        1
2                    https://briefingday.com/m/v4n3i4f3        1
3      https://briefingday.com/n/20200618/m#commentform        0
4                           https://briefingday.com/fan        1
...                                                 ...      ...
2994  https://www.smartcitiesworld.net/news/news/dee...        0
2995                      https://www.youtube.com/watch        1
2996  https://techcrunch.com/2019/07/04/an-optimisti...        0
2997  https://www.technologyreview.com/2019/12/20/13...        0
2998       https://www.bbc.com/news/technology-51018758        0

[2999 rows x 2 columns]>

In [8]:
df_raw['url'].sample(20)

777         https://www.voxmedia.com/legal/privacy-notice
1912    https://www.currentaffairs.org/2020/06/the-col...
228     https://www.theverge.com/2020/6/20/21295380/ap...
653     https://www.fool.com/the-ascent/credit-cards/l...
451     https://thehill.com/homenews/administration/50...
1913                   https://benjaminreinhardt.com/wddw
146     https://www.theguardian.com/commentisfree/2020...
421     https://www.vox.com/the-goods/2020/6/22/212989...
497     https://www.vox.com/the-goods/2020/6/22/212989...
2904    https://www.si.com/sports-illustrated/2020/06/...
2341    https://diversyfund.com/lp/growth-reit-platfor...
443     https://www.espn.com/racing/nascar/story/_/id/...
370     https://www.morningbrew.com/daily/stories/2020...
2157    https://subscribe.washingtonpost.com/newslette...
2663    https://docs.google.com/forms/d/e/1FAIpQLSew3g...
670     https://www.politico.com/states/florida/story/...
2462                               https://www.float.com/
805           

In [9]:
df_raw['is_spam'].value_counts()

0    2303
1     696
Name: is_spam, dtype: int64

In [10]:
df_raw['is_spam'].value_counts()
print("Hay spam: ",len(df_raw.loc[df_raw.is_spam==1]))
print("No son spam : ",len(df_raw.loc[df_raw.is_spam==0]))
print(df_raw.shape)

Hay spam:  696
No son spam :  2303
(2999, 2)


In [11]:
df_raw = df_raw.drop_duplicates()
df_raw = df_raw.reset_index(inplace = False)[['url','is_spam']]
df_raw.shape

(2369, 2)

In [12]:
df_raw['url'] = df_raw['url'].str.lower()

In [13]:
nltk.download('stopwords')
nltk.download('punkt')

stopwords = nltk.corpus.stopwords.words('english')
punctuation = string.punctuation

print(stopwords[:5])
print(punctuation)

[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/gitpod/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['i', 'me', 'my', 'myself', 'we']
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [14]:
df_aux = df_raw.copy()

In [15]:
limpieza = []

for p in range(len(df_aux.url)):
    desc = df_aux['url'][p]
    
    #savar la puntuacion
    desc = re.sub('[^a-zA-Z]', ' ', desc)
    
    #borrar etiquetas especiales
    desc=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",desc)
    
    #borrar digitos y caracteres especiales
    desc=re.sub("(\\d|\\W)+"," ",desc)
    
    limpieza.append(desc)

#assign the cleaned descriptions to the data frame
df_aux['url'] = limpieza
        
df_aux.head()

,url,is_spam
0,https briefingday us list manage com unsubscribe,1
1,https www hvper com,1
2,https briefingday com m v n i f,1
3,https briefingday com n m commentform,0
4,https briefingday com fan,1


In [16]:
df_aux['url'].str.split(expand=True).stack().value_counts()[:60]

https          2336
com            2065
www            1512
the             354
html            296
news            274
a               252
us              248
to              218
of              173
coronavirus     172
e               150
org             146
c               136
article         131
b               124
in              115
and             113
morningbrew     105
story           105
nytimes         101
on              101
daily            99
d                98
stories          94
utm              90
for              90
youtube          89
v                89
trump            88
numlock          87
watch            86
f                78
is               77
new              76
p                69
world            68
substack         68
reuters          65
covid            63
s                62
briefingday      61
index            61
vox              59
en               59
articles         58
cnn              58
iduskbn          58
politics         56
co               56


In [17]:
stop_words = ['http','www','com','you','your','for','not','have','is','in','im','from','to','https','e','c','v','b','f','p']

In [18]:
def remove_stopwords(message):
  if message is not None:
    words = message.strip().split()
    words_filtered = []
    for word in words:
      if word not in stop_words:
        words_filtered.append(word) 
    result = " ".join(words_filtered)         
  else:
    result = None

  return result

In [19]:
df_aux['url']=df_aux['url'].apply(remove_stopwords)

In [20]:
df_aux.sample(10)

,url,is_spam
1548,texastribune org face mask order dallas harris...,0
1796,helpcenter washingtonpost hc en us,1
1520,podcasts apple us podcast foxy browns id,0
1213,kareemali bandcamp album black advancement,0
2185,theguardian world jun coronavirus world map wh...,0
2135,buzzfeednews article tanyachen youtube has sus...,0
2368,bbc news technology,0
229,politico gallery trump rally tulsa oklahoma ph...,0
1639,masslive sports college football vs mississipp...,0
254,npr org sections health shots younger adults a...,0


In [21]:

df_aux['url'].str.split(expand=True).stack().value_counts()[:60]

the            354
html           296
news           274
a              252
us             248
of             173
coronavirus    172
org            146
article        131
and            113
story          105
morningbrew    105
on             101
nytimes        101
daily           99
d               98
stories         94
utm             90
youtube         89
trump           88
numlock         87
watch           86
new             76
world           68
substack        68
reuters         65
covid           63
s               62
index           61
briefingday     61
vox             59
en              59
cnn             58
iduskbn         58
articles        58
politics        56
co              56
n               54
cnbc            54
sunday          51
business        49
court           48
apnews          47
u               46
email           46
facebook        46
health          45
be              41
supreme         41
bbc             41
blog            40
are             40
black       

In [22]:
df = df_aux.copy()

In [23]:
X = df['url']
y = df['is_spam']
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,random_state=121)

In [24]:
vec = CountVectorizer(stop_words='english')
X_train = vec.fit_transform(X_train).toarray()
X_test = vec.transform(X_test).toarray()

In [26]:
print(X_train)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [27]:
X_train.shape

(1776, 4825)

In [28]:
X_test.shape

(593, 4825)

In [29]:
print(vec.get_feature_names_out())

['aa' 'aab' 'aaron' ... 'zulalimtm' 'zwift' 'zwn']


In [30]:
nb = MultinomialNB()

In [31]:
nb.fit(X_train, y_train)

MultinomialNB()

In [32]:
nb.score(X_train,y_train)

0.956081081081081

In [33]:
nb.score(X_test,y_test)

0.9173693086003373

In [34]:
predictions = nb.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.93      0.98      0.96       532
           1       0.68      0.38      0.48        61

    accuracy                           0.92       593
   macro avg       0.80      0.68      0.72       593
weighted avg       0.91      0.92      0.91       593



In [35]:

message_vectorizer = CountVectorizer().fit_transform(df['url'])

# Haciendo el split de los datos

X_train, X_test, y_train, y_test = train_test_split(message_vectorizer, df['is_spam'], test_size = 0.40, random_state = 121, shuffle = True)

In [36]:
cl = svm.SVC(C=1.0, kernel='linear', degree=4, gamma='auto')

In [37]:
cl.fit(X_train, y_train)
pred = cl.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.95      0.98      0.96       847
           1       0.72      0.54      0.62       101

    accuracy                           0.93       948
   macro avg       0.84      0.76      0.79       948
weighted avg       0.92      0.93      0.92       948



In [38]:
pickle.dump(cl, open('/workspace/NLP/models/texto_NLP.pkl', 'wb'))

In [39]:

load_model = pickle.load(open('/workspace/NLP/models/texto_NLP.pkl', 'rb'))

In [40]:
load_model

SVC(degree=4, gamma='auto', kernel='linear')